# Plotting a LandSat tile

---

### What does this cover?
1.  Opening LandSat data using `geopandas`
2.  

In [1]:
import xarray
import cartopy
import matplotlib.pyplot as plt
import numpy as np
import cmocean
import geopandas

Downloaded L1 LandSat images from Amazon Web Services:  
https://landsatonaws.com/L8/226/057/LC08_L1GT_226057_20180815_20180815_01_RT

In [ ]:
!ls ../data-files/LC08_L1GT_226057_20180815_20180815_01_RT_B2.TIF

In [ ]:
# band 2 (blue)
L8_b2 = xarray.open_rasterio('../data-files/LC08_L1GT_226057_20180815_20180815_01_RT_B2.TIF')
# band 3 (green)
L8_b3 = xarray.open_rasterio('../data-files/LC08_L1GT_226057_20180815_20180815_01_RT_B3.TIF')
# band 4 (red)
L8_b4 = xarray.open_rasterio('../data-files/LC08_L1GT_226057_20180815_20180815_01_RT_B4.TIF')

In [ ]:
L8_b2

In [ ]:
plt.imshow(L8_b2[0,:,:], origin='upper')

In [ ]:
plt.imshow(L8_b3[0,:,:], origin='upper')

In [ ]:
plt.imshow(L8_b4[0,:,:], origin='upper', cmap='gist_earth', vmin=50, vmax=30000)
plt.colorbar()

In [ ]:
data_extent = (L8_b2.transform[0], \
               L8_b2.transform[0] + L8_b2.x.size*L8_b2.transform[1], \
               L8_b2.transform[3] + L8_b2.y.size*L8_b2.transform[5], \
               L8_b2.transform[3])

Now merge RGB channels into one and plot a "true color" image.  Adapted from here:  
https://colekrehbiel.wordpress.com/2017/10/01/creating-natural-color-rgb-composites-from-landsat-8-data-in-python/

In [ ]:
def norm(band):
    band[band<0]=0
    band[band>20000]=20000
    band_min, band_max = 0, 20000 #band.min(), band.max()
    return ((band - band_min)/(band_max - band_min))

In [ ]:
L8_b2_norm = norm(L8_b2[0,:,:].values)
L8_b3_norm = norm(L8_b3[0,:,:].values)
L8_b4_norm = norm(L8_b4[0,:,:].values)

Download shoreline data from:  
https://www.ngdc.noaa.gov/mgg/shorelines/

Or download OpenStreetMap coastlines data from:  
http://openstreetmapdata.com/data/coastlines

In [ ]:
#shapefile = geopandas.read_file('../data-files/gshhg-shp-2.3.7/GSHHS_shp/f/GSHHS_f_L2.shp')
shapefile = geopandas.read_file('../data-files/coastlines-split-4326/lines.shp')

In [ ]:
shapefile.head()

In [ ]:
L8_rgb_432 = numpy.dstack((L8_b4_norm, L8_b3_norm, L8_b2_norm))

In [ ]:
#modis_globe = cartopy.crs.Globe(datum='WGS84', ellipse='sphere', semimajor_axis=6371007.181)
map_proj = cartopy.crs.epsg(32622)
data_proj = cartopy.crs.epsg(32622)

fig = plt.figure(figsize=(4.25,4.25))
ax = fig.add_subplot(111, projection=map_proj)
image = ax.imshow(L8_rgb_432, extent=(data_extent[0],data_extent[1],data_extent[2],data_extent[3]), \
                  origin='upper', transform=data_proj)
ax.coastlines(resolution='10m', color='firebrick')

fig.tight_layout()
fig.savefig('../figures/5-landsat-true-color.png', bbox_inches='tight', transparent=True, dpi=300)